In [2]:
# Need newer torch to use all augmentation functions
%pip install torch==2.0.* torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 35.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 18.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 9.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 18.9 MB/s eta 0:00:00
  Installing build 

In [ ]:
import os
os._exit(00)

In [1]:
import glob
import os
import multiprocessing
import random

from itertools import repeat, product

from collections import namedtuple
from csv import writer

import torch
import torchaudio
import pandas as pd
import tqdm

import tqdm.contrib.itertools as tqdm_itertools
from IPython.display import Audio
from datasets import Audio as DSAudio, load_dataset

tqdm_chain = tqdm_itertools.itertools.chain

import os
import glob
import re
import shutil

import pandas as pd

In [2]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_source_base = os.path.join(base_dir, 'gdsc_data/data_source')
data_processed_base = os.path.join(base_dir, 'gdsc_data/data_processed')

SOURCE_SAMPLING_RATE = 44100
MODEL_SAMPLING_RATE = 32000

DATA_TAG = 'data_8c86715'

In [3]:
val_data_source_path = os.path.join(data_source_base, 'val/')

In [46]:
# df = pd.read_csv(val_data_source_path + 'metadata.csv')
# df_new_validation = df.groupby('label', group_keys=False).apply(lambda x: x.sample(1))
# key_diff = list(set(df.index).difference(df_new_validation.index))

In [4]:
dest_dir = os.path.join(data_processed_base, DATA_TAG, '02_resample_validation')
dest_dir_val = os.path.join(dest_dir, 'val/')
dest_dir_train = os.path.join(dest_dir, 'train/')

In [66]:
# # raise, if not created - fools protection
# os.stat(dest_dir)

# os.makedirs(dest_dir_train)

# for filename in df_to_train['file_name']:
#     src = os.path.join(val_data_source_path, filename)
#     dest = os.path.join(dest_dir_train, filename)
#     shutil.copy(src, dest)
    
# df_to_train.to_csv(os.path.join(dest_dir_train, 'metadata.csv'), index=False)

# os.makedirs(dest_dir_val)

# for filename in df_new_validation['file_name']:
#     src = os.path.join(val_data_source_path, filename)
#     dest = os.path.join(dest_dir_val, filename)
#     shutil.copy(src, dest) 
    
# df_new_validation.to_csv(os.path.join(dest_dir_val, 'metadata.csv'), index=False)

In [5]:
DataPoint = namedtuple('DataPoint', ['path', 'label', 'name'])

In [6]:
rir_library_path = os.path.join(base_dir, 'audio_data/rir_library_flac_sampled/')

In [7]:
def resample_source_wav(data: DataPoint, resampler):
    wav, _ = torchaudio.load(data.path)
    return resampler(wav)    


def iter_datapoint_from(source_data_path):
    metadata_file = os.path.join(source_data_path, 'metadata.csv')
    df_audio = pd.read_csv(metadata_file)
    for fname, label in df_audio.itertuples(index=False):
        full_path = os.path.join(source_data_path, fname)
        yield DataPoint(path=full_path, label=label, name=fname)
        
        
def rir_path_to_name_path(rir_path):
    name, ext = os.path.splitext(os.path.basename(rir_path))
    return name, rir_path

In [8]:
def worker_init(worker_exec, source_dir, processed_dir):
    import torch
    import torchaudio
    torch.set_num_threads(1)
    
    worker_exec.torch = torch
    worker_exec.torchaudio = torchaudio
    worker_exec.source_dir = source_dir
    worker_exec.processed_dir = processed_dir
    worker_exec.resampler = torchaudio.transforms.Resample(SOURCE_SAMPLING_RATE, MODEL_SAMPLING_RATE)
    
    
def worker_main(rir_name, rir_path, data_point):
    # Load initalized libraries in a process
    torch = worker_main.torch
    torchaudio =  worker_main.torchaudio
    processed_dir = worker_main.processed_dir
    source_dir = worker_main.source_dir
    resampler = worker_main.resampler
    
    # convert to 32kHZ
    wav_resampled = resample_source_wav(data_point, resampler)
    
    # get the room IR, normalize it, then apply
    rir, _ = torchaudio.load(rir_path)
    rir = rir / torch.norm(rir, p=2)
    wav_resampled = torchaudio.functional.fftconvolve(wav_resampled, rir)
    
    # prepare for output
    source_path = data_point.path
    dp_label = data_point.label    
    dp_name = data_point.name
    source_name, source_ext = os.path.splitext(dp_name)
    
    dest_name = f'{source_name}@conv_{rir_name}.flac'
    dest_relpath = os.path.dirname(os.path.relpath(source_path, source_dir))
    dest_fpath = os.path.join(processed_dir, dest_relpath, dest_name)  
    
    os.makedirs(os.path.dirname(dest_fpath), exist_ok=True)
    
    torchaudio.save(
        dest_fpath,
        wav_resampled,
        sample_rate=MODEL_SAMPLING_RATE,
        format="flac", 
        bits_per_sample=24)

    return dest_fpath, dest_name, dp_label 


def worker_starmap(args):
    return worker_main(*args)

In [18]:
RIR_SAMPLE_SIZE = 20

rir_library = list(map(rir_path_to_name_path, glob.glob(os.path.join(rir_library_path, '**/*.flac'), recursive=True)))
source_library = iter_datapoint_from(dest_dir_train)

all_combinations = product(random.sample(rir_library, RIR_SAMPLE_SIZE), source_library)

source_library_length = iter_datapoint_from(dest_dir_train)
all_combinations_length = 0
for i, _ in enumerate(product(range(0, RIR_SAMPLE_SIZE), source_library_length), 1):
    all_combinations_length = i

work_generator = ((rir_name, rir_path, dp) for (rir_name, rir_path), dp in all_combinations)

In [19]:
print(f"Total possible rir+sound combinations {all_combinations_length}")

Total possible rir+sound combinations 10260


In [25]:
dest_dir2 = os.path.join(data_processed_base, DATA_TAG, '03_apply_ir_function_on_validation/')
dest_dir2_train = os.path.join(dest_dir2, 'train/')

In [22]:
# fools protection
os.stat(dest_dir2)

os.stat_result(st_mode=16877, st_ino=3647971464753303429, st_dev=59, st_nlink=4, st_uid=0, st_gid=0, st_size=6144, st_atime=1689003971, st_mtime=1689003971, st_ctime=1689003971)

In [23]:
print(f"Reading files from '{dest_dir_train}'")
print(f"Going to write into '{dest_dir2_train}'")

Reading files from '/root/data/gdsc_data/data_processed/data_8c86715/02_resample_validation/train/'
Going to write into '/root/data/gdsc_data/data_processed/data_8c86715/02_apply_ir_function_on_validation/train/'


In [24]:
n_cpu = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=n_cpu, initializer=worker_init, initargs=(worker_main, dest_dir_train, dest_dir2_train))

rows = []
for _, dest_fname, label in tqdm.tqdm(pool.imap_unordered(worker_starmap, work_generator), total=all_combinations_length):
    rows.append((dest_fname, label))

del pool

df = pd.DataFrame(rows, columns=['file_name', 'label'])
df.to_csv(os.path.join(dest_dir2_train, 'metadata.csv'), index=False)

100%|██████████| 10260/10260 [35:08<00:00,  4.87it/s] 


In [ ]:
## The process crashed - not enough memory :(
## Need to recreate csv file with metadata....

In [26]:
def extract_original_filename(file_name):
    parts = file_name.split("@")
    if len(parts) == 1:
        return file_name
    
    return f"{parts[0]}.wav"

In [65]:
# # files = glob.glob(os.path.join(dest_dir2_train, '*.flac'))
# df = pd.DataFrame(map(lambda x: os.path.basename(x), files), columns=["file_name"])
# df["original_filename"] = df["file_name"].map(extract_original_filename)

In [66]:
# df_source = pd.read_csv(dest_dir_train + '/metadata.csv').set_index("file_name")

In [67]:
# df = df.join(df_source, on="original_filename", how="inner")

In [27]:
df

,file_name,label
0,Chorthippusbrunneus_XC751398-dat022-008_edit5@...,10
1,Atrapsaltacorticina_GBIF2901504947_IN62966536_...,3
2,Omocestusviridulus_XC752263-dat012-007_edit1@c...,39
3,Conocephalusfuscus_XC752526-dat060-004_edit3@c...,17
4,Cyclochilaaustralasiae_GBIF1978841250_IN191825...,18
...,...,...
10255,Grylluscampestris_GBIF3772468767_IN115095988_4...,29
10256,Yoyettacelis_GBIF2873801193_IN61269567_139618@...,64
10257,Chorthippusbrunneus_XC751372-dat001-023@conv_S...,10
10258,Decticusverrucivorus_XC751968-dat016-014_edit1...,19


In [28]:
df["original_filename"] = df["file_name"].map(extract_original_filename)

In [29]:
df_grouped = df.groupby("original_filename").aggregate(
    {
        "file_name": lambda x: ", ".join(x),
        "label": min
    }
).reset_index().rename({"file_name": "file_name"})

In [30]:
df_grouped.to_csv(dest_dir2 + '/metadata_grouped_train.csv', index=False)

In [75]:
df_grouped

,original_filename,file_name,label
0,Achetadomesticus_XC751734-dat001-055_edit1.wav,Achetadomesticus_XC751734-dat001-055_edit1@con...,0
1,Achetadomesticus_XC751734-dat001-055_edit2.wav,Achetadomesticus_XC751734-dat001-055_edit2@con...,0
2,Achetadomesticus_XC751734-dat001-055_edit3.wav,Achetadomesticus_XC751734-dat001-055_edit3@con...,0
3,Achetadomesticus_XC751734-dat001-055_edit4.wav,Achetadomesticus_XC751734-dat001-055_edit4@con...,0
4,Achetadomesticus_XC751736-dat001-057.wav,Achetadomesticus_XC751736-dat001-057@conv_LX48...,0
...,...,...,...
508,Yoyettarepetens_GBIF1978820758_IN19037335_2829...,Yoyettarepetens_GBIF1978820758_IN19037335_2829...,65
509,Yoyettarepetens_GBIF1978855326_IN19219004_2860...,Yoyettarepetens_GBIF1978855326_IN19219004_2860...,65
510,Yoyettarepetens_GBIF2005327021_IN20313139_3116...,Yoyettarepetens_GBIF2005327021_IN20313139_3116...,65
511,Yoyettarepetens_GBIF2465318314_IN36334636_5169...,Yoyettarepetens_GBIF2465318314_IN36334636_5169...,65


In [73]:
df.to_csv(dest_dir2 + '/metadata_train.csv', index=False)